# Retriving recent tweets (API v2) about Russia x Ucrain conflict 

In [1]:
#Import necessary libraries
import os
import json
import requests
import csv
import datetime
import dateutil.parser
import unicodedata
import time
import pandas as pd 

In [2]:
#Save bearer as enviroment variable
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAPqDZwEAAAAAF664z4duoQxBeus0szmwqCFkfGM%3Dir5fHELJVaCJKsJftHoW3kMIJbqiOwJSIkB5J5AaIl9U0Yn6r7'

In [3]:
#Function to get the baerer
def auth():
    return os.getenv('TOKEN')

In [4]:
#Authentication function to access API
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [5]:
#Request function for retrive tweets
def create_url(keyword, max_results = 30):
    
    #Endepoint to get only recent tweets (7 days ago)
    search_url = "https://api.twitter.com/2/tweets/search/recent" 

    #Query paramns definition
    query_params = {'query': keyword,
                    'max_results': max_results,
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [6]:
#Function to connect code with endpoint
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token  
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [7]:
#Params passing
bearer_token = auth()
headers = create_headers(bearer_token)
#Get tweets in portuguese with the word "guerra" AND the word "russia" OR "ucrania"
keyword = "guerra (russia OR ucrania) lang:pt"
max_results = 30

In [8]:
#Get API response
url = create_url(keyword, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [9]:
#Print JSON response file (test)
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "data": [
        {
            "author_id": "1357083102909255680",
            "conversation_id": "1500002441714147329",
            "created_at": "2022-03-05T06:57:05.000Z",
            "id": "1500002441714147329",
            "lang": "pt",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 986
            },
            "referenced_tweets": [
                {
                    "id": "1499837323294527503",
                    "type": "retweeted"
                }
            ],
            "reply_settings": "everyone",
            "source": "Twitter for Android",
            "text": "RT @hoc111: R\u00fassia coloca leis ainda mais restritas sufocando a liberdade de express\u00e3o\n\nAt\u00e9 chamar de \"guerra\" t\u00e1 proibido\n\nFacebook e Twit\u2026"
        },
        {
            "author_id": "1395340702318485504",
            "conversation_id": "150000235

# Saving data in CSV file

In [10]:
# Create CSV file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers, passing the data that will be write into the file
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

In [11]:
#Function to write data into the file
def write_to_csv(json_response, fileName):
    
    counter = 0

    #Open OR create CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet over requests
    for tweet in json_response['data']:
        
        #Desirable data variables definition
        author_id = tweet['author_id']

        created_at = dateutil.parser.parse(tweet['created_at'])

        
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        
        tweet_id = tweet['id']

        lang = tweet['lang']

        
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        
        source = tweet['source']

        
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter)

In [12]:
write_to_csv(json_response, "data.csv")

# of Tweets added from this response:  30
